In [29]:
import pathlib
import torch
import numpy as np
from torch.utils.data import Dataset
try:
    import pyspng
    PYSPNG_IMPORTED = True
except ImportError:
    PYSPNG_IMPORTED = False
    print("Could not load pyspng. Defaulting to pillow image backend.")
    from PIL import Image


class FDF256Dataset(Dataset):

    def __init__(self,
                 dirpath: str,
                 load_keypoints: bool = False,
                 img_transform: torch.nn.Module = None,
                 load_masks: bool = False,
                 mask_transform: torch.nn.Module = None,
                 load_impath: bool = False):
        dirpath = pathlib.Path(dirpath)
        self.dirpath = dirpath
        self.img_transform = img_transform
        self.mask_transform = mask_transform
        self.load_masks = load_masks
        self.load_keypoints = load_keypoints
        self.load_impath = load_impath
        assert self.dirpath.is_dir(),\
            f"Did not find dataset at: {dirpath}"
        image_dir = self.dirpath.joinpath("images")
        self.image_paths = list(image_dir.glob("*.png"))
        assert len(self.image_paths) > 0,\
            f"Did not find images in: {image_dir}"
        self.image_paths.sort(key=lambda x: int(x.stem))
        self.landmarks = np.load(self.dirpath.joinpath("landmarks.npy")).reshape(-1, 7, 2).astype(np.float32)
        self.bounding_boxes = torch.from_numpy(np.load(self.dirpath.joinpath("bounding_box.npy")))
        assert len(self.image_paths) == len(self.bounding_boxes)
        assert len(self.image_paths) == len(self.landmarks)
        print(
            f"Dataset loaded from: {dirpath}. Number of samples:{len(self)}")

    def get_mask(self, idx):
        mask = torch.ones((1, 256, 256), dtype=torch.bool)
        bounding_box = self.bounding_boxes[idx]
        x0, y0, x1, y1 = bounding_box
        mask[:, y0:y1, x0:x1] = 0
        return mask

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        impath = self.image_paths[index]
        print(impath)
        if PYSPNG_IMPORTED:
            with open(impath, "rb") as fp:
                im = pyspng.load(fp.read())
        else:
            with Image.open(impath) as fp:
                im = np.array(fp)
        if self.img_transform is not None:
            im = self.img_transform(im)
        masks = self.get_mask(index)
        if self.mask_transform is not None:
            masks = self.mask_transform(masks)
        landmark = self.landmarks[index]
        batch = {
            "img": im,
        }
        if self.load_masks:
            batch["mask"] = masks
        if self.load_keypoints:
            batch["keypoints"] = landmark
        if self.load_impath:
            batch['impath'] = str(impath)
            batch["face_bbox"] = self.bounding_boxes[index]
        return batch


Could not load pyspng. Defaulting to pillow image backend.


In [30]:
# from https://github.com/gitshanks/fer2013

from keras.models import model_from_json
import numpy as np
import cv2

json_file = open('/Users/balazsmorvay/Downloads/fer2013-master/fer.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/Users/balazsmorvay/Downloads/fer2013-master/fer.h5")
print("Loaded model from disk")

#setting image resizing parameters
WIDTH = 256
HEIGHT = 256
x=None
y=None
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

Loaded model from disk


In [36]:
from torch.utils.data import DataLoader
from einops import rearrange

train_dataset = FDF256Dataset(dirpath='/Users/balazsmorvay/Downloads/FDF256/train/', load_keypoints=True, load_masks=True, load_impath=False)
val_dataset = FDF256Dataset(dirpath='/Users/balazsmorvay/Downloads/FDF256/val/', load_keypoints=True, load_masks=True, load_impath=False)
dataloader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=False)

for batch in dataloader:
    full_size_image = np.array(batch['img'].squeeze())
    print(full_size_image.shape)
    gray=cv2.cvtColor(full_size_image, cv2.COLOR_RGB2GRAY)
    face = cv2.CascadeClassifier('/Users/balazsmorvay/Downloads/fer2013-master/haarcascade_frontalface_default.xml')
    faces = face.detectMultiScale(gray, 1.3  , 10)
    
    #detecting faces
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
        cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        #predicting the emotion
        yhat= loaded_model.predict(cropped_img)
        print("Emotion: "+labels[int(np.argmax(yhat))])
    
    

Dataset loaded from: /Users/balazsmorvay/Downloads/FDF256/train. Number of samples:241982
Dataset loaded from: /Users/balazsmorvay/Downloads/FDF256/val. Number of samples:6531
/Users/balazsmorvay/Downloads/FDF256/train/images/0.png
(256, 256, 3)
1/1 [==============================] - 0s 14ms/step
Emotion: Surprise
/Users/balazsmorvay/Downloads/FDF256/train/images/1.png
(256, 256, 3)
1/1 [==============================] - 0s 13ms/step
Emotion: Angry
/Users/balazsmorvay/Downloads/FDF256/train/images/2.png
(256, 256, 3)
/Users/balazsmorvay/Downloads/FDF256/train/images/3.png
(256, 256, 3)
/Users/balazsmorvay/Downloads/FDF256/train/images/4.png
(256, 256, 3)
1/1 [==============================] - 0s 13ms/step
Emotion: Sad
/Users/balazsmorvay/Downloads/FDF256/train/images/5.png
(256, 256, 3)
1/1 [==============================] - 0s 13ms/step
Emotion: Sad
/Users/balazsmorvay/Downloads/FDF256/train/images/6.png
(256, 256, 3)
/Users/balazsmorvay/Downloads/FDF256/train/images/7.png
(256, 256, 

KeyboardInterrupt: 